<a href="https://colab.research.google.com/github/kindachamma1/ComputerVision2025/blob/Exam/Exam_KindaChamma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COMPUTER VISION - EXAM**

Kinda Chamma

In [1]:
from google.colab import drive
drive.mount('/content/drive')

from skimage import data, measure
import skimage as sk
import matplotlib.pyplot as plt


Mounted at /content/drive
